# Analysis of the energy

In [9]:
from pathlib import Path
import sys
from sys import stdout
import inspect
from typing import Callable
from pycomfort.files import *
import polars as pl
from functional import seq
import pandas as pd

In [4]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = Path("/") / "data" / "docking"

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local yspecies")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local yspecies
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def read_tsv(p: Path, ignore_errors: bool = True) -> pl.DataFrame:
    return pl.read_csv(str(p), header=True, sep='\t', ignore_errors=ignore_errors)

In [12]:
p2_top_chains = output / "S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains"
print(p2_top_chains)
tprint(p2_top_chains)

/data/docking/S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
S5205Nr1-P2_IgG1Fc_H_top_30_heavy_chains
	FIXED_1_35744_H_112_energy_-265.2682.dcd
	FIXED_1_35744_H_112_energy_-265.2682_log.h5
	FIXED_1_35744_H_112_energy_-265.2682_log.tsv
	FIXED_1_35744_H_112_energy_-265.2682_results.chk
	FIXED_1_35744_H_1_energy_-254.21349.dcd
	FIXED_1_35744_H_1_energy_-254.21349_log.h5
	FIXED_1_35744_H_1_energy_-254.21349_log.tsv
	FIXED_1_35744_H_1_energy_-254.21349_results.chk
	FIXED_1_35744_H_64_energy_-263.5477.dcd
	FIXED_1_35744_H_64_energy_-263.5477_log.h5
	FIXED_1_35744_H_64_energy_-263.5477_log.tsv
	FIXED_1_35744_H_64_energy_-263.5477_results.chk


In [15]:
top_1_1 = read_tsv(p2_top_chains / "FIXED_1_35744_H_112_energy_-265.2682_log.tsv")
top_1_1.head(10)

"#""Step""",Potential Energy (kJ/mole),Kinetic Energy (kJ/mole),Total Energy (kJ/mole),Temperature (K)
i64,f64,f64,f64,f64
1000,-32645.351562,13343.088787,-19302.262776,262.415945
2000,-30749.363281,15312.150038,-15437.213243,301.141092
3000,-30504.910156,16028.023749,-14476.886408,315.220041
4000,-30705.404297,15425.209053,-15280.195244,303.364601
5000,-30697.541016,15866.096137,-14831.444879,312.035442
6000,-30520.982422,16023.225565,-14497.756857,315.125676
7000,-30546.253906,15948.296416,-14597.9574,313.652059
8000,-31099.132812,15876.741776,-15222.391036,312.244808
9000,-30442.289062,15514.853421,-14927.435641,305.1276
